In [1]:
###
#   AUTHOR:   Jacob L. Miller
#   DATE:     September 21, 2023
#   PROJECT:  Page Visits Funnel
###

In [2]:
import pandas as pd

Import all the files

In [3]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [4]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [5]:
# left merge visits df with cart df to get data of users who did
# and did not add a shirt to their cart
visits_cart = pd.merge(visits, cart, how='left')

Step 3: How long is `visits_cart`?

In [6]:
# determine number of users who visited the website
print(visits_cart.shape[0])

2000


Step 4: How many timestamps are null for `cart_time`?

In [7]:
print(visits_cart.head())

# determine number of users who did not add a shirt to their cart
num_only_visited = visits_cart[visits_cart['cart_time'].isnull()].shape[0]
print(num_only_visited)

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time  
0                 NaT  
1 2017-01-26 14:44:00  
2 2017-08-20 08:31:00  
3                 NaT  
4                 NaT  
1652


Step 5: What percentage only visited?

In [8]:
# determine percentage of users who visited the website but
# did not add a shirt to their cart
pct_only_visited = num_only_visited / visits_cart.shape[0] * 100
pct_only_visited = round(pct_only_visited * 100) / 100
print('Percentage of users who visited the website but didn\'t add anything to their cart: {}%'.format(pct_only_visited))

Percentage of users who visited the website but didn't add anything to their cart: 82.6%


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [9]:
# left-merge cart df with checkout df to generate
# df to see which users added something to their cart
# and went to checkout as well as those who did not go
# to checkout
cart_to_checkout = pd.merge(cart, checkout, how='left')
print(cart_to_checkout.head(), '\n')

# determine number of users who added something to their
# cart but did not make it to checkout
num_no_checkout = cart_to_checkout[cart_to_checkout['checkout_time'].isnull()].shape[0]

# determine percentage of users who added something to their
# cart but did not make it to checkout
pct_no_checkout = num_no_checkout / cart_to_checkout.shape[0] * 100
pct_no_checkout = round(pct_no_checkout * 100) / 100
print('Percentage of users who added a shirt to their cart but did not checkout: {}%'.format(pct_no_checkout))

                                user_id           cart_time  \
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00   
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00   
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00   
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00   
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00   

        checkout_time  
0 2017-11-07 21:14:00  
1                 NaT  
2 2017-03-04 11:04:00  
3 2017-09-27 08:26:00  
4                 NaT   

Percentage of users who added a shirt to their cart but did not checkout: 35.06%


Step 7: Merge it all together

In [10]:
# combine all dataframes to get full list of information on users who have 
# been on the website in any capacity
all_data = visits_cart.merge(cart_to_checkout, how='left').merge(purchase, how='left')
print(all_data.head(10))

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   
5  7bc62113-d51d-4e6b-85e0-1cf29e3de74a 2017-06-03 20:05:00   
6  6f22cd44-bc63-4449-a9af-2651859da2f9 2017-03-18 23:09:00   
7  49a6399b-c095-4e42-92eb-af90abe2bb6a 2017-01-10 12:09:00   
8  653c095d-fd74-40dd-ae3b-5a69a2613629 2017-06-27 19:21:00   
9  3d9fdac1-aae3-4a01-b448-934d12047468 2017-09-24 06:43:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
3                 NaT                 NaT         

Step 8: % of users who got to checkout but did not purchase

In [11]:
# left-merge checkout df with purchase df to generate
# df to see which users made it to checkout and made
# a purchase as well as those who did not make a 
# purchase
checkout_to_purchase = pd.merge(checkout, purchase, how='left')
print(checkout_to_purchase.head(), '\n')

# determine number of users who made it to checkout but
# did not make a purchase
num_no_purchase = checkout_to_purchase[checkout_to_purchase['purchase_time'].isnull()].shape[0]
print('Number of users who made it to checkout but did not make a purchase: {}'.format(num_no_purchase))

# determine percentage of users who made it to checkout 
# but did not make a purchase
pct_no_purchase = num_no_purchase / checkout_to_purchase.shape[0] * 100
pct_no_purchase = round(pct_no_purchase * 100) / 100
print('Percentage of users who made it to checkout but did not make a purchase: {}%'.format(pct_no_purchase))

                                user_id       checkout_time  \
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00   
1  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00   
2  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00   
3  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00   
4  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00   

        purchase_time  
0 2017-06-25 09:49:00  
1 2017-06-25 09:56:00  
2 2017-06-25 09:57:00  
3                 NaT  
4                 NaT   

Number of users who made it to checkout but did not make a purchase: 82
Percentage of users who made it to checkout but did not make a purchase: 24.55%


Step 9: check each part of the funnel, let's print all 3 of them again

In [12]:
# print out the statistics for each stage of the website's funnel 
# to determine which stage is the weakest

# Seeing as how nearly 83% of users who visited the website didn't go
# to the next stage, while each other stage saw at least 65% 
# of the users move on to the next stage, it is quite apparent that 
# the first stage is the weakest
print('Percentage of users who visited the website but didn\'t add anything to their cart ............. {}%'.format(pct_only_visited))
print('Percentage of users who added something to their cart but didn\'t make it to checkout .......... {}%'.format(pct_no_checkout))
print('Percentage of users who went to checkout but didn\'t make a purchase ........................... {}%'.format(pct_no_purchase))

Percentage of users who visited the website but didn't add anything to their cart ............. 82.6%
Percentage of users who added something to their cart but didn't make it to checkout .......... 35.06%
Percentage of users who went to checkout but didn't make a purchase ........................... 24.55%


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [13]:
# add column to all_data df to hold the time for each 
# user to make a purchase from the time they visit
# the website
all_data['time_to_purchase'] = all_data['purchase_time'] - all_data['visit_time']

Step 11: examine the results

In [14]:
print(all_data['time_to_purchase'][~all_data['time_to_purchase'].isnull()].head(10))

1    0 days 00:44:00
14   0 days 00:38:00
48   0 days 00:41:00
49   0 days 01:00:00
65   0 days 01:00:00
70   0 days 00:34:00
71   0 days 00:35:00
72   0 days 00:53:00
75   0 days 00:36:00
76   0 days 00:25:00
Name: time_to_purchase, dtype: timedelta64[ns]


Step 12: average time to purchase

In [15]:
# calculate the average time it takes a user to make
# a purchase from the moment they visit the website
mean_purchase_time = all_data['time_to_purchase'].mean()
print(mean_purchase_time)

0 days 00:43:12.380952380
